In [1]:
from tinyphysics import run_rollout_controller
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
import numpy as np
from controllers.pidff import Controller




c:\Users\pmazu\Documents\GitHub\controls_challenge\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def optimization_function(params):
    controller = Controller(params)
    data_paths = [f'./data/{i:05}.csv' for i in [0, 5, 6]]
    stored_costs = []
    def rollout(data_paths):
        costs = [run_rollout_controller(data_path, controller=controller, model_path='./models/tinyphysics.onnx') for data_path in data_paths]
        cost = sum([cost[0]['total_cost'] for cost in costs]) / len(costs)
        return cost
    last_cost = rollout(data_paths)
    stored_costs.append(last_cost)
    if (last_cost <= 100):
        print("params", params)
        data_paths = [f'./data/{i:05}.csv' for i in range(6, 10)]
        last_cost = rollout(data_paths)
        stored_costs.append(last_cost) 
    return np.mean(stored_costs)


In [5]:
def bayesian_optimize():
    space = [
        Real(0.0, 10, name='p_base'),
        Real(0.0, 1.0, name='i_base'),
        Real(-1, 1, name='d_base'),
        Integer(0.0, 1, name='future_feedforward_weight'),
    ]
    x0 = [0.001, 0.10626976158279121, 0.0848612290131688,
                0.0]

    result = gp_minimize(optimization_function,
                            space, n_calls=300, random_state=42, verbose=True, n_initial_points=30, x0=x0)




    # Update the PID parameters with the optimized values
    print(f"Optimized parameters: {result.x}")
    print(f"Optimized cost: {result.fun}")

In [6]:
bayesian_optimize()

Iteration No: 1 started. Evaluating function at provided point.
params [0.001, 0.10626976158279121, 0.0848612290131688, 0.0]
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 4.0794
Function value obtained: 84.2932
Current minimum: 84.2932
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.7112
Function value obtained: 13712.2946
Current minimum: 84.2932
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.7257
Function value obtained: 13281.2778
Current minimum: 84.2932
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.7047
Function value obtained: 56131.4354
Current minimum: 84.2932
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.6902
Function 